In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
# %% Function

def market_capture_ratio(returns):
    """
    Function to calculate the upside and downside capture for a given set of returns.
    The function is set up so that the investment's returns are in the first column of the dataframe
    and the index returns are the second column.
    :param returns: pd.DataFrame of asset class returns
    :return: pd.DataFrame of market capture results
    """

    # initialize an empty dataframe to store the results
    df_mkt_capture = pd.DataFrame()

    # 1) Upside capture ratio
    # a) Isolate positive periods of the index
    up_market = returns[returns.iloc[:, -1] >= 0]

    # b) Geometrically link the returns
    up_linked_rets = ((1 + up_market).product(axis=0)) - 1

    # c) Calculate the ratio, multiply by 100 and round to 2 decimals to show in percent
    up_ratio = (up_linked_rets / up_linked_rets.iloc[-1] * 100).round(2)

    # 2) Downside capture ratio
    # a) Isolate negative periods of the index
    down_market = returns[returns.iloc[:, -1] < 0]

    # b) Geometrically link the returns
    down_linked_rets = ((1 + down_market).product(axis=0)) - 1

    # c) Calculate the ratio, multiply by 100 and round to 2 decimals to show in percent
    down_ratio = (down_linked_rets / down_linked_rets.iloc[-1] * 100).round(2)

    # 3) Combine to produce our final dataframe
    df_mkt_capture = pd.concat([up_ratio, down_ratio], axis=1)

    df_mkt_capture.columns = ['Upside Capture', 'Downside Capture']

    return df_mkt_capture

# %% Retrieve the returns

# Specify the tickers to retrieve using yfinance
tickers = ['FCNTX', 'SPY']
start_date = '2014-06-01'
end_date = '2024-05-31'

# Retrieve the historical data for the tickers
df_prices = yf.download(tickers, start=start_date, end=end_date)

# Keep only the adjusted close columns
df_prices = df_prices['Adj Close']

# Resample to month end and calculate the monthly percent change
df_rets_monthly = df_prices.resample('M').last().pct_change().dropna()

# Calculate the market capture ratios
df_mkt_capture = market_capture_ratio(df_rets_monthly)

[*********************100%***********************]  2 of 2 completed


C:\Users\Wonyoung\AppData\Roaming\Python\Python310\site-packages\yfinance\utils.py:768: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\Wonyoung\AppData\Roaming\Python\Python310\site-packages\yfinance\utils.py:768: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\Wonyoung\AppData\Local\Temp\ipykernel_27196\4082315792.py:56: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_rets_monthly = df_prices.resample('M').last().pct_change().dropna()


In [3]:
df_mkt_capture

,Upside Capture,Downside Capture
FCNTX,118.37,99.25
SPY,100.00,100.00
